In [467]:
# !pip install pandas openpyxl

### SET UP FOLDERS

In [468]:
import pandas as pd
import numpy as np
import os
# # Directory containing the Excel files
directory = 'C:/Users/huynh/attila'
# combined_filepath = os.path.join(directory, 'combined_excel.xlsx')
# combined_filepath_edited = os.path.join(directory, 'combined_excel_edited.xlsx')
combined_filepath_csv = os.path.join(directory, 'combined_df.csv')
parameters_filepath = os.path.join(directory, 'parameters.xlsx')
mini_df_filepath_csv = os.path.join(directory, 'mini_df.csv')
# mini_df_filepath_excel = os.path.join(directory, 'mini_df.xlsx')
combined_edited_filepath_csv = os.path.join(directory, 'combined_df_edited.csv')
df_list_pickle_filepath = os.path.join(directory, 'df_list_pickle.pkl')

minimize_data = True

### IMPORT DATA AND CLEAN

In [469]:
### COMBINE EXCELS INTO ONE - FILL COLUMNS

# # List to hold the dataframes
# dfs = []

# # Loop through each file in the directory
# for filename in os.listdir(directory):
#     if filename.endswith('.xlsx'):
#         # Read the Excel file into a dataframe
#         filepath = os.path.join(directory, filename)
#         df = pd.read_excel(filepath, engine='openpyxl')
        
#         # Append the dataframe to the list
#         dfs.append(df)

# # Combine the dataframes vertically
# combined_df = pd.concat(dfs, ignore_index=True)

# # Fill blank cells with the value from the cell above for columns DG onwards
# columns_to_fill = combined_df.columns.get_loc('Tags')
# combined_df.iloc[:, columns_to_fill:] = combined_df.iloc[:, columns_to_fill:].fillna(method='ffill')

# # Save the combined dataframe to a new Excel file in the same directory
# combined_df.to_excel(combined_filepath, index=False, engine='openpyxl')

# print(f"Combined Excel file saved at {combined_filepath}")


In [470]:
### SAVE DF TO CSV 
# combined_df.to_csv(combined_filepath_csv, index=False)

In [471]:
# ### READ CSV TO DF
# combined_df_old = pd.read_csv(combined_filepath_csv)
# combined_df_old.shape

In [472]:
# ### UNIQUE TAGS BEFORE:
# len(np.unique(combined_df_old['Tags']))

# count = combined_df_old['Tags'].str.contains('segment', case=False, na=False).sum()

# # Print the count
# print(f'Number of rows where column X contains "segment": {count}')

In [473]:
# ### READ PARAMETERS EXCEL 

# import re  # Add this line to import the 're' module

# parameters = pd.read_excel(parameters_filepath, engine='openpyxl')
# newname = parameters.iloc[:, 0].astype(str).values
# oldname = parameters.iloc[:, 1].astype(str).values

# len(parameters)
# newname

# ### CREATE COLUMN THAT IS JUST THE LAST TEXT IN TAGS

# tags_split = combined_df['Tags'].str.split(',')

# last_non_numeric_name = tags_split.apply(lambda x: next((n.strip() for n in reversed(x) if not any(c.isdigit() for c in n) and "Manual" not in n and "Figure" not in n), None))
# combined_df['Tags_last_non_numeric'] = last_non_numeric_name

# ### CREATE ANOTHER COLUMN FOR "DOUBLE"
# name_containing_double = tags_split.apply(lambda x: next((n.strip() for n in reversed(x) if "Double" in n), None))
# combined_df['Tags_double'] = name_containing_double


# ### CREATE NEW COLUMN BASED ON LAST TEXT IN TAGS
# combined_df['Tagsnew'] = ''

# for old, new in zip(oldname, newname):
#     combined_df.loc[(combined_df['Tags_double'].str.endswith(old, na=False)) & (combined_df['Tagsnew'] == ''), 'Tagsnew'] = new
#     combined_df.loc[(combined_df['Tags_last_non_numeric'].str.endswith(old, na=False)) & (combined_df['Tagsnew'] == ''), 'Tagsnew'] = new


In [474]:
# ### CHECK DISTRIBUTION OF NEW TAGS
# tagsnew_counts = combined_df['Tagsnew'].value_counts()
# print(tagsnew_counts)
# sum_of_counts = tagsnew_counts.sum()
# unique_tags_count = combined_df['Tagsnew'].nunique()


# print("Sum of all counts in 'Tagsnew' column:", sum_of_counts)
# print("No empty column in 'Tagsnew' column:", sum_of_counts==combined_df.shape[0])
# print("Count of unique tags:", unique_tags_count)
# print("All tags used:", unique_tags_count==len(newname))


# # ERROR CHECK:  Get the DataFrame containing rows where Tagsnew is empty
# empty_tags = combined_df.loc[combined_df['Tagsnew'] == '', 'Tags']

# # Iterate over each row in the Series
# for index, tags in empty_tags.items():
#     print(f"Row {index}: Tags = {tags}")


In [538]:
### IMPORT CSV TO DF

combined_df_newtags_csv = os.path.join(directory, 'combined_df_newtags.csv')
combined_df = pd.read_csv(combined_df_newtags_csv)
print(f'original df shape {combined_df.shape}')

### EDIT SOME columns:  

# FIX MID
combined_df['MID'] = combined_df['MID'].astype(str) ## some values are read as int -- change everything to text

# FIX Z
combined_df['z_new'] = (480 - combined_df['Z, Center of Geometry (µm)'] - (480 - combined_df['Z2, Bounding Box (µm)'])) * -1 ## edit request by Attila (30 May)

# ADDING ROWS FOR POST NOSTIM

# Step 1: Filter rows where Treatment is 'NoSTIM' and Condition is 'pre'
df_nostim = combined_df[(combined_df['Treatment'] == 'NoSTIM') & (combined_df['Condition'] == 'Pre')]

# CHECK 
print(f"{combined_df[(combined_df['Treatment'] == 'NoSTIM')].shape[0] == df_nostim.shape[0]}")

row_count_nostim_post = combined_df[(combined_df['Treatment'] == 'NoSTIM') & (combined_df['Condition'] == 'Post')].shape[0]

if row_count_nostim_post == 0:
    # Step 2: Duplicate the filtered rows
    df_duplicated = df_nostim.copy()

    # Step 3: Change 'Condition' values from 'pre' to 'post' in the duplicated rows
    df_duplicated['Condition'] = df_duplicated['Condition'].replace('Pre', 'Post')

    # Step 4: Append the duplicated rows back to the original DataFrame
    combined_df = pd.concat([combined_df, df_duplicated], ignore_index=True)

# # CREATE CELL ID:  CHECK with attila, this seems unnecessary since I'm filtering by mouse id in my table, so counting unique cells should still work.   
# combined_df['Id'] = combined_df['Id'].astype(str)
# combined_df['id_mid'] = combined_df.apply(lambda row: row['Id'] + '_' + row['MID'], axis=1)
# ## CHECK 

# Remove columns
combined_df = combined_df[['MID', 'Id', 'z_new', 'Treatment','Tagsnew','Condition','Experimental day','Area (Mesh), Projection (XY/Z) (µm²)']]

# Remove tags
combined_df = combined_df[~combined_df['Tagsnew'].isin(['anode', 'cathode', 'bubbles'])]

print(f'new df shape {combined_df.shape}')



C:\Users\huynh\AppData\Local\Temp\ipykernel_22732\2368298274.py:4: DtypeWarning:

Columns (43,44,111) have mixed types. Specify dtype option on import or set low_memory=False.



original df shape (675825, 132)
True
new df shape (862489, 8)


In [517]:
unique_z_new_per_id_mid = combined_df.groupby('id_mid')['z_new'].nunique() == 1
unique_z_new_per_id_mid

id_mid
100000_FDG67477        True
100000_FDG67796       False
100000_FDG68014-1D     True
100000_FDG82699        True
100001_FDG67477        True
                      ...  
99998_FDG67477         True
99998_FDG67796        False
99999_FDG67477         True
99999_FDG67796        False
9_FDG83478-1D         False
Name: z_new, Length: 542145, dtype: bool

In [529]:
z_new_values = combined_df[combined_df['id_mid'] == '99998_FDG67477']['z_new']
z_new_values

203751   -30.833333
744226   -30.833333
Name: z_new, dtype: float64

In [503]:
### Create tag groups (from Attila Meeting 12 July)
tag_groups = {
    'GFPin':    {"DL bGFPin", "DL sGFPin", "DL bYFPin", "DL sYFPin", 
                "bGFPin", "sGFPin"},
    'GFPout':   {"DL bGFPall", "DL sGFPall", "DL bYFPall", "DL sYFPall", 
                "bGFPall", "sGFPall"},
    'YFPin':    {"bYFPin", "sYFPin"},
    'YFPout':   {"bYFPall", "sYFPall"},
    'allDLin':  {"DL bGFPin", "DL sGFPin", "DL bYFPin", "DL sYFPin"},
    'allDLout': {"DL bGFPall", "DL sGFPall", "DL bYFPall", "DL sYFPall"},
    'sGFPin':   {"DL sGFPin", "DL sYFPin", "sGFPin"},
    'sGFPout':  { "DL sGFPall", "DL sYFPall", "sGFPall", },
    'bGFPin':   {"DL bGFPin","DL bYFPin", "bGFPin"},
    'bGFPout':  {"DL bGFPall", "DL bYFPall", "bGFPall"},
    'sYFPin':   {"sYFPin"},
    'sYFPout':  {"sYFPall"},
    'bYFPin':   {"bYFPin"},
    'bYFPout':  {"bYFPall"},
    'bead':  {"bead"},
}
tag_groups_density = {key: value for key, value in tag_groups.items() if 'in' in key}

if minimize_data: 
    tag_groups = {'YFPin': {"bYFPin", "sYFPin"}}
    
n_tag_groups = len(tag_groups)  # Number of tags in tag_groups
n_tag_groups_density = len(tag_groups_density)  # Number of tags in tag_groups



In [536]:
#### READ AREA FILE (ATTILA 30 May)

area_excel_path = os.path.join(directory, '1-2-3-5-4-M.xlsx')
columns_to_import = list(range(1, pd.read_excel(area_excel_path, nrows=0).shape[1], 4))
df_area = pd.read_excel(area_excel_path, usecols=columns_to_import)

area_lists = [df_area[col].tolist() for col in df_area.columns]
n_area_lists = len(area_lists)
n_area_planes = len(area_lists[0]) 
# print(df_area)

### Get names:
import re

column_names = df_area.columns
# Split the column names by _ or -
split_column_names = [re.split('[_-]', name) for name in column_names]

# Find the maximum number of words in any column name
max_words = max(len(words) for words in split_column_names)

# Initialize lists to hold words by their position
word_lists = [[] for _ in range(max_words)]

# Populate the word lists
for words in split_column_names:
    for i, word in enumerate(words):
        word_lists[i].append(word)

# # Print the lists
# for i, word_list in enumerate(word_lists, start=0):
#     print(f"Word count {len(word_list)}, at position {i}:", word_list)
# print()
# Print the unique words
for i, word_list in enumerate(word_lists, start=0):
    print(f"Word count {len(np.unique(word_list))}, at position {i}:", np.unique(word_list))
print()
# print(split_column_names)

### MAKE PARAMETERS NAMES MATCH ORIGINAL PARAMETERS 

# Iterate over the list and apply the replacements
area_mids = []
for item in word_lists[1]:
    if "5440" in item.lower():
        item = "5440"
    if "68014" in item.lower():
        item = "FDG68014-1D"
    if "83478" in item.lower():
        item = "FDG83478-1D"
    area_mids.append(item)
    # print(f"Original: {item}")  # Debug print to see the replacement process
print("Unique area mids:", np.unique(area_mids))
print("Unique count:", len(np.unique(area_mids)))
print("Total count:", len(area_mids))
# print("Percent match:", len(set(area_mids).intersection(set(mids))), 'out of', n_mids)
print()

area_days = [int(text.replace("D", "")) for text in word_lists[4]]
print("Unique area days:", np.unique(area_days))
print("Total count:", len(area_days))
print()

area_conditions = []
for item in word_lists[5]:
    if "no" in item.lower() or "wrong" in item.lower() or "before" in item.lower():
        item = "Pre"
    if "after" in item.lower():
        item = "Post"
    area_conditions.append(item)
    # print(f"Original: {item}")  # Debug print to see the replacement process
print("Unique area conditions:", np.unique(area_conditions))
print("Total count:", len(area_conditions))
print()

area_treatments = word_lists[2]
area_treatments[45:51] = ['NoSTIM'] * 6 ### IMPORTANT:  editing column names to match treatment from planewise_xp.xls (EDIT CONFIRMED BY ATTILA via whatsapp, 5 June 2024)
print("Unique area treatments:", np.unique(area_treatments))
# print("Percent match:", len(set(area_treatments).intersection(set(treatments))), 'out of', n_treatments)
print()

### DUPLICATE ROWS WITH PRE NOSTIM
# find index from treatment list containing 'no'

n_rows_original = 51
nostim_indices = [index for index, value in enumerate(area_treatments) if value == "NoSTIM"]

if len(area_days) == n_rows_original:
    for index in nostim_indices:
        # print(index)
        area_days.append(area_days[index])
        area_conditions.append("Post")
        area_treatments.append(area_treatments[index])
        area_mids.append(area_mids[index])
        area_lists.append(area_lists[index])

n_area_lists = n_rows_original+len(nostim_indices)

print(f'new area_days count: {len(area_days)}')
print(f'new area_conditions count: {len(area_conditions)}')
print(f'new area_mids count: {len(area_mids)}')
print(f'new area_treatments count: {len(area_treatments)}')
print(f'new area_lists count: {len(area_lists)}')
print(f'new n_area_lists: {n_area_lists}')



Word count 1, at position 0: ['Area']
Word count 12, at position 1: ['FDG5020' 'FDG5025' 'FDG5411' 'FDG5440' 'FDG67477' 'FDG67796'
 'FDG68014GBM' 'FDG82338' 'FDG82341' 'FDG82699' 'FDG83478GBM' 'FDG83480']
Word count 3, at position 2: ['100Hz' '1Hz' 'NoSTIM']
Word count 7, at position 3: ['1' '2' '4' '5' '6' '7' '8']
Word count 3, at position 4: ['D26' 'D29' 'D33']
Word count 6, at position 5: ['NoStim' 'afterstim' 'beforestim' 'noStim' 'nostim' 'wrongEIS']
Word count 1, at position 6: ['nostim']
Word count 1, at position 7: ['ctr']

Unique area mids: ['5440' 'FDG5020' 'FDG5025' 'FDG5411' 'FDG67477' 'FDG67796' 'FDG68014-1D'
 'FDG82338' 'FDG82341' 'FDG82699' 'FDG83478-1D' 'FDG83480']
Unique count: 12
Total count: 51

Unique area days: [26 29 33]
Total count: 51

Unique area conditions: ['Post' 'Pre']
Total count: 51

Unique area treatments: ['100Hz' '1Hz' 'NoSTIM']

new area_days count: 63
new area_conditions count: 63
new area_mids count: 63
new area_treatments count: 63
new area_lists 

In [479]:
# ### CHECK AREA_MIDS CONTAINS ALL 3 DAYS.    

# unique_ages_dict = {}

# for name, age in zip(area_mids, area_days):
#     if name not in unique_ages_dict:
#         unique_ages_dict[name] = set()
#     unique_ages_dict[name].add(age)

# unique_ages_dict


In [480]:
# ### CONFIRM THAT ALL 51 AREA PARAMETERS CAPTURE ALL 600k ROWS in DF

# from collections import Counter

# count_total = 0
# index_lists = []

# # Iterate through the data
# for area_mid, area_day, area_condition, area_treatment in zip(area_mids, area_days, area_conditions, area_treatments):
#     # Append list of matching row indexes to index_lists
#     index_list = list(combined_df[(combined_df['Condition'] == area_condition) & 
#                                   (combined_df['Treatment'] == area_treatment) & 
#                                   (combined_df['MID'] == area_mid) & 
#                                   (combined_df['Experimental day'] == area_day)].index)
#     index_lists.append(index_list)
#     count_total += len(index_list)
#     # print(f"Count of rows: {count_total}")

# # Flatten the index_lists into a single list
# flattened_index_lists = [index for sublist in index_lists for index in sublist]

# # Use Counter to count the occurrences of each index
# index_counts = Counter(flattened_index_lists)

# # Initialize counters for 0, 1, or more appearances
# count_0 = 0
# count_1 = 0
# count_more = 0

# # Find the maximum index in the index_lists to account for all potential indexes
# all_indexes = range(max(flattened_index_lists) + 1)

# # Count occurrences
# for index in all_indexes:
#     count = index_counts[index]
#     if count == 0:
#         count_0 += 1
#     elif count == 1:
#         count_1 += 1
#     else:
#         count_more += 1
# print(count_0, ' ', count_1, " ", count_more, ' total match', count_1==combined_df.shape[0])


In [481]:
# ### FOR FINDING ERRORS

# max_index = max(flattened_index_lists) if flattened_index_lists else 0
# # Find indexes that appear 0 times
# indexes_0_times = [index for index in range(max_index + 1) if index_counts[index] == 0]
# # Print the results
# print(f"Indexes appearing 0 times: {indexes_0_times}")

# area_row_test = 39

# for row in indexes_0_times[:3]:
# # Print the specified columns
#     combined_df.iloc[row]
#     print("Condition:", combined_df.iloc[row]['Condition'], ' || match to area: ', combined_df.iloc[row]['Condition']==area_conditions[area_row_test])
#     print("Treatment:", combined_df.iloc[row]['Treatment'], ' || match to area: ', combined_df.iloc[row]['Treatment']==area_treatments[area_row_test], " ", area_treatments[area_row_test])
#     print("MID:", combined_df.iloc[row]['MID'], ' || match to area: ', combined_df.iloc[row]['MID']==area_mids[area_row_test], " ", area_mids[area_row_test])
#     print("Experimental day:", combined_df.iloc[row]['Experimental day'], ' || match to area: ', combined_df.iloc[row]['Experimental day']==area_days[area_row_test], " || ", area_treatments[area_row_test])
#     print()

In [482]:
# EXPORT the DataFrame to a CSV file and EXCEL

# mini_df.to_csv(mini_df_filepath_csv, index=False)
# mini_df.to_excel(mini_df_filepath_excel, index=False)
# print("mini_df exported successfully.")

# combined_df.to_csv(combined_edited_filepath_csv, index=False)
# print("combined_df_edited exported successfully.")

In [483]:
# ### OPTIONAL:  SHRINK DF (BY ROWS AND/OR COLUMNS)

### REDUCE COLUMNS
# # List column names of the DataFrame
# column_names = combined_df.columns.tolist()
# print("Column names of the DataFrame:", column_names)

# columns_to_keep = ['Type', 'Tags', 'MID', 'Experimental day', 'Z, Center of Geometry (µm)', 'Condition', 'Treatment', 'Tags_last_non_numeric', 'Tagsnew']

# # Keep only the specified columns
# mini_df = mini_df.loc[:, columns_to_keep]


In [484]:
tags

array(['bYFPin'], dtype='<U6')

In [540]:
if minimize_data:
    mini_tags = ['bYFPin'] # ['cathode', 'anode', 'bead', 'bubbles', 'sYFPin', 'DL sGFPin',
    #    'sYFPall', 'bYFPall', 'sGFPall', 'sGFPin', 'DL sGFPall', 'bGFPall',
    #    'DL bGFPall', 'GBM', 'bYFPin', 'DL sYFPall', 'DL bGFPin', 'bGFPin',
    #    'DL bYFPall', 'DL sYFPin', 'DL bYFPin']
    mini_df = combined_df[combined_df['Tagsnew'].str.contains('|'.join(mini_tags))]
    # mini_df = combined_df.sample(n=50000, random_state=42)  # random_state for reproducibility
    df = mini_df
    print(f"df is minimized")
    print(f'row count: {df.shape[0]}')
    print()
else:
    df = combined_df

### PULL KEY PARAMETERS FROM DF
tags = df['Tagsnew'].unique()
if minimize_data:
    tags = np.array(mini_tags)
n_tags = len(tags)
print(n_tags, " tags: ", tags)
print()

conditions = df['Condition'].unique()
n_conditions = len(conditions)
print(n_conditions , " conditions: ", conditions)
print()

treatments = df['Treatment'].unique()
n_treatments = len(treatments)
print(n_treatments, " treatments: ", treatments)
print()


### OLD z_bin code:
z_bin_size = 10 #### CHANGED 30 May, per Attila
z_max = int(df['z_new'].max())
z_min = int(df['z_new'].min())
z_min_rounded = ((z_min - 9) // 10) * 10  # Round z_min up to the nearest multiple of 10
z_max_rounded = ((z_max + 9) // 10) * 10  # Round z_max up to the nearest multiple of 10
z_bins_upper = list(range(z_min_rounded, 0+z_bin_size, z_bin_size))[::-1] ### INVERSE to match how area, which starts at the top plane (0), and goes deeper 
# z_bins_upper = list(range(n_area_planes*z_bin_size*-1, 0+z_bin_size, z_bin_size))[::-1] ### Z goes from zero to negative 300, so z_min is actually most negative number.
n_z_bins = len(z_bins_upper)-1
print("z: ", z_max, ' to ', z_min)
print("z_bins_upper: ", z_bins_upper) ### Larger than z max because of area 
print("Number of bins: ", n_z_bins)

print()
mids = df['MID'].unique()
n_mids = len(mids)
print(n_mids, " MIDs: ", mids)
print()

days = df['Experimental day'].unique()
days = np.array(sorted(days))
n_days = len(days)
print(n_days, " experimental days: ", days)
print()

cids = df['Id'].unique()
cids = np.array(sorted(cids))
n_cids = len(cids)
print(n_cids, " cell ids: ", cids)
print()


df is minimized
row count: 13257

1  tags:  ['bYFPin']

2  conditions:  ['Post' 'Pre']

3  treatments:  ['1Hz' '100Hz' 'NoSTIM']

z:  -14  to  -181
z_bins_upper:  [0, -10, -20, -30, -40, -50, -60, -70, -80, -90, -100, -110, -120, -130, -140, -150, -160, -170, -180, -190]
Number of bins:  19

12  MIDs:  ['FDG5020' 'FDG5025' 'FDG5411' '5440' 'FDG67477' 'FDG67796' 'FDG68014-1D'
 'FDG82338' 'FDG82341' 'FDG82699' 'FDG83478-1D' 'FDG83480']

3  experimental days:  [26 29 33]

10320  cell ids:  [ 11834  11836  11842 ... 675065 675069 675080]



In [498]:
# # Example z_bin_size. This can be changed to any value from 10 to 100.
# z_bin_size = 10  # Change this value as needed

# area_bin_size = int(z_bin_size/10)

# z_max = int(df['z_new'].max())
# z_min = int(df['z_new'].min())
# z_min_rounded = ((z_min - (z_bin_size - 1)) // z_bin_size) * z_bin_size  # Round z_min up to the nearest multiple of z_bin_size
# z_max_rounded = ((z_max + (z_bin_size - 1)) // z_bin_size) * z_bin_size  # Round z_max up to the nearest multiple of z_bin_size

# # Generate the bins starting from 0 and moving downwards with steps of z_bin_size
# z_bins_upper = list(range(0, z_min_rounded-z_bin_size, -z_bin_size))  # Note: z_min_rounded is already negative

# n_z_bins = len(z_bins_upper) -1

# print("z: ", z_max, ' to ', z_min)
# print("z_bins_upper: ", z_bins_upper)  # Larger than z max because of area
# print("Number of bins: ", n_z_bins)


z:  -14  to  -181
z_bins_upper:  [0, -10, -20, -30, -40, -50, -60, -70, -80, -90, -100, -110, -120, -130, -140, -150, -160, -170, -180, -190]
Number of bins:  19


In [488]:
sum(area_list[2:4])

1987878.8169999998

In [489]:
area_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
n_bins = int(len(area_list) // area_bin_size)
area_list_binned = []
for i in range(n_bins):
    start_index = i * area_bin_size
    end_index = start_index + area_bin_size
    group_sum = sum(area_list[start_index:end_index])
    area_list_binned.append(group_sum)

print(area_list_binned)


[6, 15, 24]


In [490]:
# # ## CHECK THAT NOSTIM HAS PRE AND POST 
# print(df[(df['Treatment'] == 'NoSTIM') & (df['Condition'] == 'Post')].shape[0])
# print(df[(df['Treatment'] == 'NoSTIM') & (df['Condition'] == 'Pre')].shape[0])  


In [491]:
### COLORS FOR ALL PLOTS
color_list = [
    'rgba(0, 0, 255, 1)',  # blue
    'rgba(0, 128, 0, 1)',  # green
    'rgba(255, 0, 0, 1)',  # red
    'rgba(255, 165, 0, 1)',  # orange
    'rgba(128, 0, 128, 1)',  # purple
    'rgba(0, 255, 255, 1)',  # cyan
    # Add more colors if needed
]

### Functions

In [723]:
import pandas as pd
import numpy as np

def calculate_avg_from_df_list(df_list):  ### intakes a list of dataframes, stacks them, and outputs mean, std, and count.  
    df_stacked = pd.concat(df_list, keys=range(len(df_list)))

    # Calculate the average and standard deviation
    df_stacked_avg = df_stacked.groupby(level=1).mean()
    df_stacked_std = df_stacked.groupby(level=1).std()
    df_stacked_count = df_stacked.groupby(level=1).count()

    return df_stacked_avg, df_stacked_std, df_stacked_count

def merge_df_list(dfs):

def merge_df_list(dfs):
    # Check if input is a non-empty list of DataFrames
    if not isinstance(dfs, list) or len(dfs) == 0 or not all(isinstance(df, pd.DataFrame) for df in dfs):
        raise ValueError("Input must be a list of non-empty DataFrames.")
    
    # Initialize the merged DataFrame with the same index and columns as the first DataFrame
    df1 = dfs[0]
    merged_df = pd.DataFrame(index=df1.index, columns=df1.columns)
    
    for col in df1.columns:
        for idx in df1.index:
            merged_list = []
            for df in dfs:
                cell_value = df.at[idx, col]
                if not (isinstance(cell_value, float) and pd.isna(cell_value)):  # Check for NaN values
                    if isinstance(cell_value, list):
                        merged_list.extend(cell_value)
                    else:
                        merged_list.append(cell_value)
            merged_df.at[idx, col] = merged_list
    
    return merged_df


In [ ]:
# ### TEST merge_df_list

# import pandas as pd

# # Example DataFrames
# df1 = df_soads_d_itcm[0][0][0][0]
# df2 = df_soads_d_itcm[0][0][0][11]
# df3 = df_soads_d_itcm[0][0][0][2]

# dfs = [df1, df2, df3]

# def merge_df_list(dfs):
#     # Check if input is a non-empty list of DataFrames
#     if not isinstance(dfs, list) or len(dfs) == 0 or not all(isinstance(df, pd.DataFrame) for df in dfs):
#         raise ValueError("Input must be a list of non-empty DataFrames.")
    
#     # Initialize the merged DataFrame with the same index and columns as the first DataFrame
#     df1 = dfs[0]
#     merged_df = pd.DataFrame(index=df1.index, columns=df1.columns)
    
#     for col in df1.columns:
#         for idx in df1.index:
#             merged_list = []
#             for df in dfs:
#                 cell_value = df.at[idx, col]
#                 if not (isinstance(cell_value, float) and pd.isna(cell_value)):  # Check for NaN values
#                     if isinstance(cell_value, list):
#                         merged_list.extend(cell_value)
#                     else:
#                         merged_list.append(cell_value)
#             merged_df.at[idx, col] = merged_list
    
#     return merged_df

# # Merge the DataFrames
# merged_df = merge_df_list(dfs)
# print(merged_df)

In [493]:
### PLOT FUNCTION (WITHOUT MOUSE COUNT)

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def plot_lines_columns_rows(data_type, plot_data, tag, lines, columns, rows):  # day lines, treatment columns, condition rows
    i = np.where(tags == tag)[0][0]  # tag_list_id

    if 'den' in data_type: 
        title_text = "Density (cells/area)"
    elif 'soad' in data_type: 
        title_text = "SOAD"
    else:
        title_text = "Data type unknown"
    # Create a subplot figure with 2 rows and len(treatments) columns
    fig = make_subplots(rows=len(rows), cols=len(columns), subplot_titles=[
        f"{tag} | {column} | {row}" 
        for row in rows
        for column in columns
    ])

    # Loop through each row and each column to plot its data
    for row_num, row in enumerate(rows, start=1):
        idx1 = np.where(rows == row)[0][0]  # condition_list_id
                        
        for col_num, column in enumerate(columns, start=1):
            idx2 = np.where(columns == column)[0][0]  # treatment_list_id

            if not plot_data[i][idx2][idx1]:
                print(f'EMPTY: {tag}, {column}, {row}')
                continue

            avg = plot_data[i][idx2][idx1][0]
            std = plot_data[i][idx2][idx1][1]
            count = plot_data[i][idx2][idx1][2]
            
            # Define colors for each line
            colors = {line: color_list[idx % len(color_list)] for idx, line in enumerate(lines)}
                        
            # Add lines and error bands for the given treatment
            for col in avg.columns:
                color = colors.get(col, 'rgba(0, 0, 0, 1)')  # Default color if not in colors dict
                fillcolor = color.replace('1)', '0.2)')  # Make the color transparent
                
                # Add the average line
                fig.add_trace(go.Scatter(
                    y=avg.index,
                    x=avg[col],
                    mode='lines',
                    name=f'{col}',
                    line=dict(color=color)
                ), row=row_num, col=col_num)
                
                # Add the upper bound of the error band
                fig.add_trace(go.Scatter(
                    y=avg.index,
                    x=avg[col] + std[col],
                    mode='lines',
                    line=dict(color='rgba(0,0,0,0)'),
                    showlegend=False,
                    hoverinfo='skip'
                ), row=row_num, col=col_num)
                
                # Add the lower bound of the error band
                fig.add_trace(go.Scatter(
                    y=avg.index,
                    x=avg[col] - std[col],
                    mode='lines',
                    fill='tonextx',  # Fill the area between the two traces
                    fillcolor=fillcolor,
                    line=dict(color='rgba(0,0,0,0)'),
                    showlegend=False,
                    hoverinfo='skip'
                ), row=row_num, col=col_num)
            
            # Update subplot layout 
            fig.update_xaxes(title_text='Cells/µm²', row=row_num, col=col_num, matches='x')
            fig.update_yaxes(title_text='Z (µm)', row=row_num, col=col_num, matches='y')

    # Update layout for the entire figure
    fig.update_layout(
        height=800,  # Increased height to accommodate 2 rows
        width=1200,
        title_text=title_text,
        template='plotly_white'
    )

    # Show the plot
    fig.show()



In [494]:
### PLOT FUNCTION V2 with mouse count
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import inspect

def plot_lines_columns_rows_test(data_type, plot_data, tag, lines, columns, rows):
    i = np.where(tags == tag)[0][0]  # tag_list_id

    if 'den' in data_type: 
        title_text = "Density (cells/area)"
    elif 'soad' in data_type: 
        title_text = "SOAD"
    else:
        title_text = "Data type unknown"
    
    # Create a subplot figure with len(rows) rows and 2*len(columns) columns (extra columns for tables)
    specs = []
    for _ in range(len(rows)):
        row_specs = []
        for _ in range(len(columns)):
            row_specs.extend([{'type': 'xy'}, {'type': 'domain'}])
        specs.append(row_specs)

    fig = make_subplots(
        rows=len(rows), 
        cols=2*len(columns),
        column_widths=[1, 0.3] * len(columns),  # Adjusted widths for plots and tables
        horizontal_spacing=0.1,
        specs=specs
    )

    # Loop through each row and each column to plot its data
    for row_num, row in enumerate(rows, start=1):
        idx1 = np.where(rows == row)[0][0]  # condition_list_id
                        
        for col_num, column in enumerate(columns, start=1):
            idx2 = np.where(columns == column)[0][0]  # treatment_list_id

            if not plot_data[i][idx2][idx1]:
                print(f'EMPTY: {tag}, {column}, {row}')
                continue

            avg = plot_data[i][idx2][idx1][0]
            std = plot_data[i][idx2][idx1][1]
            count = plot_data[i][idx2][idx1][2]
            
            # Define colors for each line
            colors = {line: color_list[idx % len(color_list)] for idx, line in enumerate(lines)}
                        
            # Add lines and error bands for the given treatment
            for col in avg.columns:
                color = colors.get(col, 'rgba(0, 0, 0, 1)')  # Default color if not in colors dict
                fillcolor = color.replace('1)', '0.2)')  # Make the color transparent
                
                # Add the average line
                fig.add_trace(go.Scatter(
                    y=avg.index,
                    x=avg[col],
                    mode='lines',
                    name=f'{col}',
                    line=dict(color=color)
                ), row=row_num, col=2*col_num-1)
                
                # Add the upper bound of the error band
                fig.add_trace(go.Scatter(
                    y=avg.index,
                    x=avg[col] + std[col],
                    mode='lines',
                    line=dict(color='rgba(0,0,0,0)'),
                    showlegend=False,
                    hoverinfo='skip'
                ), row=row_num, col=2*col_num-1)
                
                # Add the lower bound of the error band
                fig.add_trace(go.Scatter(
                    y=avg.index,
                    x=avg[col] - std[col],
                    mode='lines',
                    fill='tonextx',  # Fill the area between the two traces
                    fillcolor=fillcolor,
                    line=dict(color='rgba(0,0,0,0)'),
                    showlegend=False,
                    hoverinfo='skip'
                ), row=row_num, col=2*col_num-1)
            
            # Create count table with flipped values
            fig.add_trace(go.Table(
                header=dict(
                    values=["<b>Count</b>"],
                    font=dict(size=10),
                    align="left"
                ),
                cells=dict(
                    values=[count.values[::-1].tolist()],  # Reverse the order of the count values
                    align="left"
                )
            ), row=row_num, col=2*col_num)
            
            # Update subplot layout 
            fig.update_xaxes(title_text='Cells/µm²', row=row_num, col=2*col_num-1, matches='x')
            fig.update_yaxes(title_text='Z (µm)', row=row_num, col=2*col_num-1, matches='y')

    # Add titles manually to ensure correct alignment
    annotations = []
    for row_num, row in enumerate(rows, start=1):
        for col_num, column in enumerate(columns, start=1):
            title = f"{tag} | {column} | {row}"
            annotations.append(dict(
                x=0.5 / len(columns) + (col_num - 1) / len(columns),
                y=1.3 - (row_num - 0.5) / len(rows),
                xref='paper',
                yref='paper',
                text=title,
                showarrow=False,
                font=dict(size=12)
            ))

    fig.update_layout(
        height=800,  # Increased height to accommodate the rows
        width=1300,
        title_text=title_text,
        template='plotly_white',
        annotations=annotations
    )

    # Show the plot
    fig.show()

# Example usage
# plot_c_d_t(df_plot_data, tag)  # Assuming df_plot_data and tag are defined


### Calculate day lines (both density and SOAD)

In [535]:
print(df.columns)


Index(['id_mid', 'other_column'], dtype='object')


In [719]:
### CREATE DF_DENSITY

import pandas as pd
import os
import pickle

tbl_d_lines = pd.DataFrame(index=z_bins_upper, columns=days) ### Create empty tables (dataframes) at the center of the lists of lists: day columns, z rows
### NOMENCLATURE:  d_lines describes columns of dataframe (or "lines" in the plot), so d_lines = days, t_lines = treatment columns, c_lines = conditions. (df rows are always z)

# Initialize the nested lists for storing the data
df_density_d_itcm = [[[[tbl_d_lines.copy() for _ in range(n_mids)] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tag_groups)]
df_soad_d_itcm = [[[[tbl_d_lines.copy() for _ in range(n_mids)] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tag_groups)]
df_soads_d_itcm = [[[[tbl_d_lines.copy() for _ in range(n_mids)] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tag_groups)]
df_tumor_d_itcm = [[[[tbl_d_lines.copy() for _ in range(n_mids)] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tag_groups)]

### NOMENCLATURE:  density vs soad = describes data type in df.  
### d describes column of df.  itcm describes the folder heirarchy:  
### i = tags, t = treatment, c = conditions, and finally m = mouse, which contains the df with days columns.  

row_count = 0

# Loop through each tag in tag_groups
for i, tag_group in enumerate(tag_groups):
    print(f"Working on tag group {i+1} of {n_tag_groups} ({tag_group})")
    
    for area_list_id in range(n_area_lists):
        t = np.where(treatments == area_treatments[area_list_id])[0][0]  # treatment_list_id
        c = np.where(conditions == area_conditions[area_list_id])[0][0]  # condition_list_id
        m = np.where(mids == area_mids[area_list_id])[0][0]  # mid_list_id
        day = area_days[area_list_id]

        for z, z_bin in enumerate(z_bins_upper):
            df_filtered = df[(df['Tagsnew'].isin(tag_groups[tag_group])) &
                                (df['Condition'] == area_conditions[area_list_id]) &
                                (df['Treatment'] == area_treatments[area_list_id]) &
                                (df['MID'] == area_mids[area_list_id]) &
                                (df['Experimental day'] == day) &
                                (df['z_new'] <= z_bin) &
                                (df['z_new'] > z_bin - z_bin_size)]

            # Fill density table
            object_count = df_filtered['Id'].nunique()
            area = area_lists[area_list_id][z]
            if area > 0:
                density = object_count / area
            else:
                density = 0  # Handle division by zero

            df_density_d_itcm[i][t][c][m].loc[z_bin, day] = density if not df_filtered.empty else 0

            # Fill soad tables
            mesh_max = df_filtered['Area (Mesh), Projection (XY/Z) (µm²)'].max()
            df_soad_d_itcm[i][t][c][m].loc[z_bin, day] = mesh_max if not df_filtered.empty else 0
            mesh_list = df_filtered['Area (Mesh), Projection (XY/Z) (µm²)'].tolist()
            df_soads_d_itcm[i][t][c][m].loc[z_bin, day] = mesh_list if not df_filtered.empty else np.nan

            # Fill tumor table
            df_tumor_d_itcm[i][t][c][m].loc[z_bin, day] = area

            row_count += object_count  # Increment row count


print(f'all rows accounted for: {row_count == df.shape[0]}')


Working on tag group 1 of 1 (YFPin)
all rows accounted for: False


In [701]:
##### CONVERT DF_PLOT_DATA TO AVG & STD for ALL MICE 
all_density_d_itc = [[[[] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tags)]
all_soad_d_itc = [[[[] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tags)]
all_tumor_d_itc = [[[[] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tags)]

avg_density_d_itc = [[[[] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tags)]
avg_soad_d_itc = [[[[] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tags)]
avg_tumor_d_itc = [[[[] for _ in range(n_conditions)] for _ in range(n_treatments)] for _ in range(n_tags)]


for i, tag_group in enumerate(tag_groups): ## tags or tags_test_set   
    for t, treatment in enumerate(treatments):
        for c, condition in enumerate(conditions):    
            ### fill density plot table        
            if df_density_d_itcm[i][t][c]: 
                # print(f'before: {len(df_plot_data[i][t][c])}')
                # print(f'before: {df_d_lines[i][t][c]}')
                avg_density_d_itc[i][t][c] = calculate_avg_from_df_list(df_density_d_itcm[i][t][c])
                # print(f'after: {len(df_plot_data[i][c][t])}')
                # print(f'after: {plot_d_lines[i][c][t]}')
            else:
                print(f'empty density: {tag_group}, {treatment}, {condition}')
            ### fill SOAD plot tables
            if df_soad_d_itcm[i][t][c]: 
                avg_soad_d_itc[i][t][c] = calculate_avg_from_df_list(df_soad_d_itcm[i][t][c])
            else:   
                print(f'empty SOAD: {tag_group}, {treatment}, {condition}')

            print(f'SOAD : {tag_group}, {treatment}, {condition}')
            print(f'SOAD : {i}, {t}, {c}')

            all_soad_d_itc[i][t][c] = merge_df_list(df_soads_d_itcm[i][t][c])

            ### fill tumor plot table
            avg_tumor_d_itc[i][t][c] = calculate_avg_from_df_list(df_tumor_d_itcm[i][t][c])


### PICKLE DATA

# if not minimize_data:
#     df_density_d_itc_pkl_path = os.path.join(directory, 'df_density_d_itc.pkl')
#     with open(df_density_d_itc_pkl_path, 'wb') as file:
#         pickle.dump(avg_density_d_itc, file)
#     df_soad_d_itc_pkl_path = os.path.join(directory, 'df_soad_d_itc.pkl')
#     with open(df_soad_d_itc_pkl_path, 'wb') as file:
#         pickle.dump(avg_soad_d_itc, file)
#     print(f"pkl saved")
#     df_tumor_d_itc_pkl_path = os.path.join(directory, 'df_tumor_d_itc.pkl')
#     with open(df_tumor_d_itc_pkl_path, 'wb') as file:
#         pickle.dump(avg_tumor_d_itc, file)
#     print(f"pkl saved")

SOAD : YFPin, 1Hz, Post
SOAD : 0, 0, 0


TypeError: can only concatenate list (not "int") to list

In [ ]:
### LOAD PICKL DATA
import pickle

df_density_d_itc_pkl_path = os.path.join(directory, 'df_density_d_itc.pkl')
with open(df_density_d_itc_pkl_path, 'rb') as file:
    plot_density_d_itc = pickle.load(file)

df_soad_d_itc_pkl_path = os.path.join(directory, 'df_soad_d_itc.pkl')
with open(df_soad_d_itc_pkl_path, 'rb') as file:
    plot_soad_d_itc = pickle.load(file)

### Calculate treatment lines (both density and SOAD)

In [ ]:
### CREATE Density plots, treatment lines, day columns, condition rows

import pandas as pd
import os
import pickle

tbl_t_lines = pd.DataFrame(index=z_bins_upper, columns=treatments)  ### Create empty tables (dataframes) at the center of the lists of lists: treatment columns, z rows

row_count = 0

# Initialize the nested lists for storing the data
df_density_t_idcm = [[[[tbl_t_lines.copy() for _ in range(n_mids)] for _ in range(n_conditions)] for _ in range(n_days)] for _ in range(n_tags)]
df_soad_t_idcm = [[[[tbl_t_lines.copy() for _ in range(n_mids)] for _ in range(n_conditions)] for _ in range(n_days)] for _ in range(n_tags)]

for i, tag in enumerate(tags): 
    print(f"Working on tag  {i+1} of {n_tags} ({tag})")
    for area_list_id in range(n_area_lists):
        d = np.where(days == area_days[area_list_id])[0][0]  # 
        c = np.where(conditions == area_conditions[area_list_id])[0][0]  # condition_list_id
        m = np.where(mids == area_mids[area_list_id])[0][0]  # mid_list_id
        treatment = area_treatments[area_list_id] 
        
        area_list = area_lists[area_list_id]
        n_bins = int(len(area_list) // area_bin_size)
        area_list_binned = []
        for i in range(n_bins):
            start_index = i * area_bin_size
            end_index = start_index + area_bin_size
            group_sum = sum(area_list[start_index:end_index])
            area_list_binned.append(group_sum)

        for z, z_bin in enumerate(z_bins_upper):
            # Filter the DataFrame
            # print(f"z_bin: {z_bin}")
            df_filtered = df[(df['Tagsnew'] == tag) & 
                                (df['Condition'] == area_conditions[area_list_id]) & 
                                (df['Treatment'] == area_treatments[area_list_id]) & 
                                (df['MID'] == area_mids[area_list_id]) & 
                                (df['Experimental day'] == area_days[area_list_id]) &
                                (df['z_new'] <= z_bin) & 
                                (df['z_new'] > z_bin - z_bin_size)]
            # fill density table
            object_count = df_filtered.shape[0]
            area = area_list_binned[z]
            if area > 0:
                density = object_count / area
            else:
                density = 0  
            if not df_filtered.empty:
                df_density_t_idcm[i][d][c][m].loc[z_bin, treatment] = density
            else:
                df_density_t_idcm[i][d][c][m].loc[z_bin, treatment] = 0

            # fill soad table
            mesh_avg = df_filtered['Area (Mesh), Projection (XY/Z) (µm²)'].mean()
            if not df_filtered.empty:
                df_soad_t_idcm[i][d][c][m].loc[z_bin, treatment] = mesh_avg
            else:
                df_soad_t_idcm[i][d][c][m].loc[z_bin, treatment] = 0 ### OR NONE?

print(f'all rows accounted for: {row_count == df.shape[0]}')

### STACK VALUES to get avg and std

plot_density_t_idc = [[[[] for _ in range(n_conditions)] for _ in range(n_days)] for _ in range(n_tags)]
plot_soad_t_idc = [[[[] for _ in range(n_conditions)] for _ in range(n_days)] for _ in range(n_tags)]

for i, tag in enumerate(tags): 
    for d, day in enumerate(days):
        for c, condition in enumerate(conditions):
            if df_density_t_idcm[i][d][c]: 
                plot_density_t_idc[i][d][c] = calculate_avg_from_df_list(df_density_t_idcm[i][d][c])
            else:
                print(f'empty density: {tag}, {day}, {condition}')

            if df_soad_t_idcm[i][d][c]: 
                plot_soad_t_idc[i][d][c] = calculate_avg_from_df_list(df_soad_t_idcm[i][d][c])
                print(plot_soad_t_idc)
            else:   
                print(f'empty SOAD: {tag}, {day}, {condition}')

# Save as pickle

if not minimize_data:
    df_density_t_idc_pkl_path = os.path.join(directory, 'df_density_t_idc.pkl')
    with open(df_density_t_idc_pkl_path, 'wb') as file:
        pickle.dump(plot_density_t_idc, file)

    df_soad_t_idc_pkl_path = os.path.join(directory, 'df_soad_t_idc.pkl')
    with open(df_soad_t_idc_pkl_path, 'wb') as file:
        pickle.dump(plot_soad_t_idc, file)
    print(f"pkl saved")


Working on tag  1 of 1 (bYFPin)
0


IndexError: list index out of range

In [ ]:
# ### LOAD PICKL DATA
# import pickle

# df_density_t_idc_pkl_path = os.path.join(directory, 'df_density_t_idc.pkl')
# with open(df_density_t_idc_pkl_path, 'rb') as file:
#     plot_density_t_idc = pickle.load(file)

# df_soad_t_idc_pkl_path = os.path.join(directory, 'df_soad_d_itc.pkl')
# with open(df_soad_t_idc_pkl_path, 'rb') as file:
#     plot_soad_t_idc = pickle.load(file)

### Calculate conditions lines (density and soad)

In [ ]:
# ### CREATE Density plots, treatment lines, day columns, condition rows

# import pandas as pd
# import os
# import pickle

# tbl_c_lines = pd.DataFrame(index=z_bins_upper, columns=conditions)  ### Create empty tables (dataframes) at the center of the lists of lists:  treatment columns, z rows

# df_density_c_idtm = [[[[tbl_c_lines.copy() for _ in range(n_mids)] for _ in range(n_treatments)] for _ in range(n_days)] for _ in range(n_tags)]
# df_soad_c_idtm = [[[[tbl_c_lines.copy() for _ in range(n_mids)] for _ in range(n_treatments)] for _ in range(n_days)] for _ in range(n_tags)]

# row_count = 0

# for i, tag in enumerate(tags): 
#     print(f"Working on tag  {i+1} of {n_tags} ({tag})")
#     for area_list in range(n_area_lists):
#         d = np.where(days == area_days[area_list])[0][0]  # 
#         t = np.where(treatments == area_treatments[area_list])[0][0]  # treatment_list_id
#         m = np.where(mids == area_mids[area_list])[0][0]  # mid_list_id
#         condition = area_conditions[area_list] 
#         for z, z_bin in enumerate(z_bins_upper):
#             df_filtered = df[(df['Tagsnew'] == tag) & 
#                                 (df['Condition'] == area_conditions[area_list]) & 
#                                 (df['Treatment'] == area_treatments[area_list]) & 
#                                 (df['MID'] == area_mids[area_list]) & 
#                                 (df['Experimental day'] == area_days[area_list]) &
#                                 (df['z_new'] <= z_bin) & 
#                                 (df['z_new'] > z_bin - z_bin_size)]
#             object_count = df_filtered.shape[0]
#             area = area_lists[area_list][z]
#             if area > 0:
#                 density = object_count / area
#             else:
#                 density = 0  
#             if not df_filtered.empty:
#                 df_density_c_idtm[i][d][t][m].loc[z_bin, condition] = density
#             else:
#                 df_density_c_idtm[i][d][t][m].loc[z_bin, condition] = 0

#             # fill soad table
#             mesh_avg = df_filtered['Area (Mesh), Projection (XY/Z) (µm²)'].mean()
#             if not df_filtered.empty:
#                 df_soad_c_idtm[i][d][t][m].loc[z_bin, condition] = mesh_avg
#             else:
#                 df_soad_c_idtm[i][d][t][m].loc[z_bin, condition] = 0

# print(f'all rows accounted for: {row_count == df.shape[0]}')

# ##### STACK PLOTS 

# plot_density_c_idt = [[[[] for _ in range(n_treatments)] for _ in range(n_days)] for _ in range(n_tags)]
# plot_soad_c_idt = [[[[] for _ in range(n_treatments)] for _ in range(n_days)] for _ in range(n_tags)]

# plot_count = 0

# for i, tag in enumerate(tags): 
#     for d, day in enumerate(days):
#         for t, treatment in enumerate(treatments):     
#             if df_density_c_idtm[i][d][t]: 
#                 df_stacked = calculate_avg_from_df_list(df_density_c_idtm[i][d][t])
#                 plot_density_c_idt[i][d][t] = df_stacked
#             else:
#                 print(f'empty density: {tag}, {day}, {treatment}')
#             if df_soad_c_idtm[i][d][t]: 
#                 plot_soad_c_idt[i][d][t] = calculate_avg_from_df_list(df_soad_c_idtm[i][d][t])
#             else:   
#                 print(f'empty SOAD: {tag}, {day}, {treatment}')


# # Save as pickle
# if not minimize_data:
#     df_density_c_idt_pkl_path = os.path.join(directory, 'df_density_c_idt.pkl')
#     with open(df_density_c_idt_pkl_path, 'wb') as file:
#         pickle.dump(plot_density_c_idt, file)
#     df_soad_c_idt_pkl_path = os.path.join(directory, 'df_soad_c_idt.pkl')
#     with open(df_soad_c_idt_pkl_path, 'wb') as file:
#         pickle.dump(plot_soad_c_idt, file)
#     print(f"pkl saved")

In [ ]:
### LOAD PICKL DATA
import pickle

df_density_c_idt_pkl_path = os.path.join(directory, 'df_density_c_idt.pkl')
with open(df_density_c_idt_pkl_path, 'rb') as file:
    plot_density_c_idt = pickle.load(file)

df_soad_c_idt_pkl_path = os.path.join(directory, 'df_soad_c_idt.pkl')
with open(df_soad_c_idt_pkl_path, 'rb') as file:
    plot_soad_c_idt = pickle.load(file)

#### PLOTS

In [ ]:
### TESTING PLOT FUNCTIONS
for i, tag in enumerate(tags):
    # plot_lines_columns_rows_test('density', plot_density_d_itc, tag, lines = days, columns = treatments, rows = conditions)
    # plot_lines_columns_rows_test('soad', plot_soad_d_itc, tag, lines = days, columns = treatments, rows = conditions)


    plot_lines_columns_rows_test('density', plot_density_t_idc, tag, lines = treatments, columns = days, rows = conditions)
    plot_lines_columns_rows_test('soad', plot_soad_t_idc, tag, lines = treatments, columns = days, rows = conditions)

    # plot_lines_columns_rows_test('density', plot_density_c_idt, tag, lines = conditions, columns = days, rows = treatments)
    # plot_lines_columns_rows_test('soad', plot_soad_c_idt, tag, lines = conditions, columns = days, rows = treatments)


    

In [ ]:
### PLOT FUNCTIONS
for i, tag in enumerate(tags):
    plot_lines_columns_rows('density', plot_density_d_itc, tag, lines = days, columns = treatments, rows = conditions)
    plot_lines_columns_rows('soad', plot_soad_d_itc, tag, lines = days, columns = treatments, rows = conditions)


    plot_lines_columns_rows('density', plot_density_t_idc, tag, lines = treatments, columns = days, rows = conditions)
    plot_lines_columns_rows('soad', plot_soad_t_idc, tag, lines = treatments, columns = days, rows = conditions)

    plot_lines_columns_rows('density', plot_density_c_idt, tag, lines = conditions, columns = days, rows = treatments)
    plot_lines_columns_rows('soad', plot_soad_c_idt, tag, lines = conditions, columns = days, rows = treatments)

### NOTES

#### Object count

In [ ]:
# ### PLOT VERSION 1:  OBJECT COUNT

# import pandas as pd
# import plotly.graph_objects as go

# df_plot = df_obj_count ### df_obj_count

# ### SELECT parameters ####
# select_tag = 'DL sGFPall'
# select_condition = 'Post'
# select_treatment = '1Hz'

# select_tag_id = int(np.where(tags == select_tag)[0])
# select_condition_id = int(np.where(conditions == select_condition)[0])
# select_treatment_id = int(np.where(treatments == select_treatment)[0])

# # Stack the DataFrames into a single DataFrame
# df_all_mice = df_plot[select_tag_id][select_condition_id][select_treatment_id]  # Assuming df_list[0] contains a list of DataFrames
# df_all_mice_stacked = pd.concat(df_all_mice, keys=range(len(df_all_mice)))

# # Calculate the average and standard deviation
# df_all_mice_stacked_avg = df_all_mice_stacked.groupby(level=1).mean()
# df_all_mice_stacked_std = df_all_mice_stacked.groupby(level=1).std()

# # Print the results
# print("Average:")
# print(df_all_mice_stacked_avg)
# print("\nStandard Deviation:")
# print(df_all_mice_stacked_std)

# import pandas as pd
# import plotly.graph_objects as go

# # Define colors for each line
# colors = {
#     26: 'rgba(0, 0, 255, 1)',  # blue
#     29: 'rgba(0, 128, 0, 1)',  # green
#     33: 'rgba(255, 0, 0, 1)',  # red
# }

# # Create the plot
# fig = go.Figure()

# # Add lines and error bands for each column, treating the columns as the y-axis
# for column in df_all_mice_stacked_avg.columns:
#     color = colors[column]
#     fillcolor = color.replace('1)', '0.2)')  # Make the color transparent
#     # Add the average line
#     fig.add_trace(go.Scatter(
#         y=df_all_mice_stacked_avg.index,
#         x=df_all_mice_stacked_avg[column],
#         mode='lines',
#         name=f'Day {column}',
#         line=dict(color=color)
#     ))
#     # Add the upper bound of the error band
#     fig.add_trace(go.Scatter(
#         y=df_all_mice_stacked_avg.index,
#         x=df_all_mice_stacked_avg[column] + df_all_mice_stacked_std[column],
#         mode='lines',
#         line=dict(color='rgba(0,0,0,0)'),
#         showlegend=False,
#         hoverinfo='skip'
#     ))
#     # Add the lower bound of the error band
#     fig.add_trace(go.Scatter(
#         y=df_all_mice_stacked_avg.index,
#         x=df_all_mice_stacked_avg[column] - df_all_mice_stacked_std[column],
#         mode='lines',
#         fill='tonextx',  # Fill the area between the two traces
#         fillcolor=fillcolor,
#         line=dict(color='rgba(0,0,0,0)'),
#         showlegend=False,
#         hoverinfo='skip'
#     ))

# # Update layout
# fig.update_layout(
#     title=(f'Average count (with std) for tag {select_tag}, condition {select_condition}, treatment {select_treatment}'),
#     yaxis_title='Z, Center of Geometry (µm)',
#     xaxis_title='Count',
#     template='plotly_white',
#     height=700,
#     width=700
# )

# # Show the plot
# fig.show()



In [ ]:
# ### TESTS

# tag = "DL sGFPall"
# condition = "Post"
# treatment = "1Hz"
# mid = "FDG5020"
# day = 26

# df_filtered = df[(df['Tagsnew'] == tag) & 
#                     (df['Condition'] == condition) & 
#                     (df['Treatment'] == treatment) & 
#                     (df['MID'] == mid) & 
#                     (df['Experimental day'] == day)]

# # condition = "Post"
# # treatment = "1Hz"
# # mid = "FDG5020"
# # day = 26
# # df_filtered = df[ #(df['Tagsnew'] == tag) & 
# #                     (df['Condition'] == condition) & 
# #                     (df['Treatment'] == treatment) & 
# #                     (df['MID'] == mid) & 
# #                     (df['Experimental day'] == day)]

# df_filtered.shape

# tag = "sYFPin"
# for i in range(n_area_lists):
#     df_filtered = df[#(df['Tagsnew'] == tag) & 
#                         (df['Condition'] == area_conditions[i]) & 
#                         (df['Treatment'] == area_treatments[i]) & 
#                         (df['MID'] == area_mids[i]) & 
#                         (df['Experimental day'] == area_days[i])]

#                         # (df['z_new'] >= z_bin) & 
#                         # (df['z_new'] < z_bin + z_bin_size)]
#     print(df_filtered.shape)
# # df_filtered['z_new']
# # df_filtered['Area (Mesh), Projection (XY/Z) (µm²)'].shape

# # df['z_new']